<a href="https://colab.research.google.com/github/LuckyNugget/Toxicity-Prediction-GNN-/blob/main/notebook/GNNMolTox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Core Dependencies

In [ ]:
# 1. Install RDKit (Cheminformatics engine)
!pip install -q rdkit


# 2. Install PyTorch Geometric (The GNN framework)
!pip install -q torch_geometric

# 3. Install DeepChem & PubChemPy (For the data)
!pip install deepchem pubchempy

Import required libraries

In [ ]:
#@title Import required libraries
# Core libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# RDKit for molecular handling
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem

# PyTorch and PyTorch Geometric
import torch
from torch_geometric.data import Data
from torch_geometric.datasets import MoleculeNet
from torch_geometric.utils import to_networkx
from torch_geometric.utils import to_undirected

#import pytorch libraries
import torch.nn.functional as F
from torch_geometric.nn import GATConv, global_mean_pool

# NetworkX for graph visualization
import networkx as nx

# IPython display utilities
from IPython.display import display

# Pubchem for data
import pubchempy as pcp

# Set plotting style
sns.set_context("notebook", font_scale=1.5)
sns.set_palette("Set2")

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
# 1. Download the CSV directly (Bypassing DeepChem's loader)
url = "https://deepchemdata.s3-us-west-1.amazonaws.com/datasets/tox21.csv.gz"
print(f"Downloading data from {url}...")
df = pd.read_csv(url)

# 2. Inspect the data
print(f"Loaded {len(df)} rows.")
print("Columns:", df.columns.tolist())

# The 'smiles' column has the molecule structure
# The other columns (NR-AR, SR-ARE, etc.) are the 12 toxicity labels
labels_cols = [c for c in df.columns if c != 'smiles' and c != 'mol_id']
print(f"Toxicity Tasks found: {len(labels_cols)}")


# Lesson learned
# I was unable to actually load the data because I kept running into a
# simple error where edge cases prevented me from loading in the data fully
# so I had to manually use pandas


In [ ]:
#turning the smiles into actual data that can be processed into the graph

def advanced_smiles_to_graph(smiles: str):

  # Create RDkit molecule from SMILES string
  mol = Chem.MolFromSmiles(smiles)
  if mol is None:
    raise ValueError(f"could not get moelcule")

  # Add hydrogens
  mol = Chem.AddHs(mol)

  # One hot encoding for each bond type
  bond_type_to_idx = {
      Chem.rdchem.BondType.SINGLE: 0,
      Chem.rdchem.BondType.DOUBLE: 1,
      Chem.rdchem.BondType.TRIPLE: 2,
      Chem.rdchem.BondType.AROMATIC: 3,
  }

  # Get total number of atoms
  n_atoms = mol.GetNumAtoms()

  # Intialize list to store node atom features
  node_features = []
  for atom in mol.GetAtoms():
    # Extract basic atom properoties
    atom_type = atom.GetSymbol()
    atomic_num = atom.GetAtomicNum()
    formal_charge = atom.GetFormalCharge()
    hybridization = atom.GetHybridization()
    is_aromatic = int(atom.GetIsAromatic())
    is_in_ring = int(atom.IsInRing())

    # Create one-hot encoding for common atom types
    atom_types = ['C', 'O', 'N', 'H', 'F', 'P', 'S', 'Cl', 'Br', 'I']
    atom_type_onehot = [1 if atom_type == t else 0 for t in atom_types]

    if atom_type not in atom_types:
        atom_type_onehot.append(1)  # "Other" category
    else:
        atom_type_onehot.append(0)

    # Create one-hot encoding for hybridization states
    hybridization_types = [
        Chem.rdchem.HybridizationType.SP,
        Chem.rdchem.HybridizationType.SP2,
        Chem.rdchem.HybridizationType.SP3
    ]

    hybridization_onehot = [1 if hybridization == h else 0 for h in hybridization_types]
    if hybridization not in hybridization_types:
        hybridization_onehot.append(1)  # "Other" hybridization
    else:
        hybridization_onehot.append(0)

    # Combine all atomic features into a single feature vector
    features = atom_type_onehot + [
        formal_charge,
        is_aromatic,
        is_in_ring,
        atom.GetDegree(),
        atom.GetTotalNumHs(),
        atom.GetNumRadicalElectrons()  # Number of unpaired electrons
    ] + hybridization_onehot

    # This is concatenation so the arrays all merge to gather to form a super array
    node_features.append(features)

  # Convert node features to numpy array for efficient computation
  node_features = np.array(node_features)

  # Initialize data structures for edge information
  adjacency = np.zeros((n_atoms, n_atoms))  # Adjacency matrix, prob don't need (only for visualization)
  edge_features = []                        # List to store bond features
  edge_indices = []

  # Process each bond in the molecule
  for bond in mol.GetBonds():

      # Get indices of atoms involved in the bond
      begin_idx = bond.GetBeginAtomIdx()
      end_idx = bond.GetEndAtomIdx()

      # Update adjacency matrix (symmetric for undirected graph)
      #adjacency[begin_idx, end_idx] = 1 DELETD CAUSE NOT NEEDED
      #adjacency[end_idx, begin_idx] = 1

      # Create one-hot encoding for bond type
      bond_type = bond.GetBondType()
      bond_type_onehot = np.zeros(len(bond_type_to_idx))
      if bond_type in bond_type_to_idx:
          bond_type_onehot[bond_type_to_idx[bond_type]] = 1

      # Extract additional bond properties
      is_conjugated = int(bond.GetIsConjugated())  # Whether bond is part of a conjugated system
      is_in_ring = int(bond.IsInRing())           # Whether bond is part of a ring

      # Combine all bond features
      features = np.concatenate([bond_type_onehot, [is_conjugated, is_in_ring]])

      # Add edge in both directions (undirected graph representation)
      edge_features.append(features)
      edge_indices.append((begin_idx, end_idx))

      edge_features.append(features)  # Same feature for the reverse direction
      edge_indices.append((end_idx, begin_idx))

      # ^^ Interestingly, Undirected function does not actually fully resolve this because it cannnot
      # handle edge cases where there are no edge features

  # Convert edge features to numpy array, handling empty case
  if edge_features:
      edge_features = np.array(edge_features)
  else:
      edge_features = np.empty((0, len(bond_type_to_idx) + 2))  # +2 for conjugation and ring features

  return node_features, adjacency, edge_features, edge_indices


In [ ]:
#need to now take these smiles and input them into pyg format

def smiles_to_pyg(smiles:str, label=None):

  try:
    # Get the graph representation of smiles
    node_features, adjacency, edge_features, edge_indices = advanced_smiles_to_graph(smiles)
  except ValueError:
    return None

  # Put into tesnor because it allows easier calculations
  x = torch.tensor(node_features, dtype=torch.float)

  # Convert edge indices to pytorch tensor in COO format
  # edge_index is a 2 x n row list that contains source and target nodes
  # .t() transposes the matrix and .contiguous() ensures memory layout is optimal
  edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()

  # Convert edge features to PyTorch Tensor
  edge_attr = torch.tensor(edge_features, dtype=torch.float)

  # HANDLE THE LABEL (Crucial for Training)
  y = None
  if label is not None:
    # Convert label to float tensor (handle NaNs later if needed)
    y = torch.tensor(label, dtype=torch.float).view(1, -1)

  # Create PyG Data object
  # x: node features
  # edge_index: graph connectivity
  # edge_attr: edge features
  # smiles: original SMILES string (stored for reference)
  data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y, smiles=smiles)

  return data

In [ ]:
#run to generate final training set

# Identify Label Columns
label_cols = [c for c in df.columns if c not in ['smiles', 'mol_id']]

# Process the DataFrame
print(f"Processing {len(df)} molecules ...")
data_list = []

# Gets index and row where index is 0,1,2,... and row is the actual row of values
for index, row in df.iterrows():

  # Gets the actual smiles for that specific row
  smiles = row['smiles']

  # Gets all the label values for this moelcule, replaces any missing values with -1,
  # Converts from panda series to numpy arra
  labels = row[labels_cols].fillna(-1).values

  # Takes the smiles and and the labels and outputs it.
  graph_data = smiles_to_pyg(smiles, label=labels)

  # Add to list if valid
  if graph_data is not None:
    data_list.append(graph_data)

  # Progress tracking
  if index% 1000 == 0:
    print(f"   Processed {index}...")

print(f"✅ Done! Created {len(data_list)} valid graph objects.")




In [ ]:
class ToxicityGAT(torch.nn.Module):
  def __init__(self, num_node_features, hidden_channels = 64, num_classes = 12):
    super(ToxicityGAT, self).__init__()

    #each conv has an attention mechanism

    # Layer 1: Input layer -> Hidden Layer
    # heads=4 means we use "Multi-Head Attention" (4 different views of the molecule)
    self.conv1 = GATConv(num_node_features, hidden_channels, heads=4, dropout=0.1)

    # Batch Norm acts as a "Reset" button to keep numbers stable
    self.bn1 = torch.nn.BatchNorm1d(hidden_channels * 4)

    # Layer 2: Hidden -> Hidden
    # We multiply hidden_channels by 4 because the previous layer had 4 heads
    self.conv2 = GATConv(hidden_channels * 4, hidden_channels, heads=4, dropout=0.1)
    self.bn2 = torch.nn.BatchNorm1d(hidden_channels * 4)

    # Layer 3: Hidden -> Hidden (Refining the features)
    self.conv3 = GATConv(hidden_channels * 4, hidden_channels, heads=1, dropout=0.1)

    # Output Layer: Map to the 12 toxicity tasks
    self.lin = torch.nn.Linear(hidden_channels, num_classes) #no attention mechanism

  def forward(self, x, edge_index, batch):

        # Message Passing (The Graph Layers)
        x = self.conv1(x, edge_index)
        x = self.bn1(x) # Normalize
        x = F.elu(x) # ELU is often better than ReLU for GATs
        x = F.dropout(x, p=0.2, training=self.training)

        x_in = x
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.elu(x)

        x = self.conv3(x, edge_index)

        # Global Pooling (Graph -> Vector)
        # This collapses the ~20 atoms into a single vector representing the WHOLE molecule
        x = global_mean_pool(x, batch)

        # Final Classification
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)

        return x # We return Raw Logits (we'll apply Sigmoid in the loss function)

In [ ]:
# Get the number of features from your first graph
# (This comes from your advanced_smiles_to_graph function)
num_features = data_list[0].num_node_features
print(f"Each atom has {num_features} features.")

# Instantiate the model
model = ToxicityGAT(num_node_features=num_features)
print("Model initialized successfully!")

# Test on a single batch to ensure no shape errors
from torch_geometric.loader import DataLoader

test_loader = DataLoader(data_list[:32], batch_size=32)
batch = next(iter(test_loader))

# Pass the batch through the model
output = model(batch.x, batch.edge_index, batch.batch)

print(f"Output shape: {output.shape}")
# Should be [32, 12] -> (32 molecules, 12 toxicity tasks)

In [ ]:
import torch
from torch_geometric.loader import DataLoader

# Shuffle and Split
import random
random.shuffle(data_list)

split_idx = int(len(data_list) * 0.8)
train_dataset = data_list[:split_idx]
test_dataset = data_list[split_idx:]

# Create Loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialize Model & Optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Training on: {device}")

# Get number of features from the first graph
num_features = data_list[0].num_node_features
model = ToxicityGAT(num_node_features=num_features).to(device)

# Adam is the standard optimizer for GNNs (learn more!)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)

# Reduce LR if the loss stops going down for 5 epochs
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, min_lr=0.00001)

# BCEWithLogitsLoss is required for Multi-Label Binary Classification
# (combines Sigmoid + Binary Cross Entropy for stability)
num_pos = 1  # Toxic
num_neg = 15 # Safe
weight = torch.tensor([num_neg / num_pos], dtype=torch.float).to(device)

criterion = torch.nn.BCEWithLogitsLoss(pos_weight=weight) #should look WAY more into this

In [ ]:
def train():
    model.train()
    total_loss = 0

    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()

        # Forward Pass
        out = model(batch.x, batch.edge_index, batch.batch)

        # Create Mask for Valid Labels
        # We only want to calculate loss where y != -1
        y = batch.y
        is_labeled = y != -1 # Creates a True/False matrix

        # Calculate Loss (Only on valid data)
        # We slice both 'out' and 'y' using the mask
        loss = criterion(out[is_labeled], y[is_labeled])

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)

# Setup Variables
best_val_loss = float('inf')
patience_counter = 0
patience_limit = 10  # Stop if no improvement for 10 epochs

# Running
print("Starting Training...")
for epoch in range(1, 51):
    train_loss = train()

    if train_loss < best_val_loss:
      best_val_loss = train_loss
      patience_counter = 0

      torch.save(model.state_dict(), 'best_toxicity_model.pth')

    else:
      patience_counter+=1

    # Emergency break
    if patience_counter >= patience_limit:
        print(f"🛑 Early Stopping triggered at Epoch {epoch}!")
        break


    scheduler.step(train_loss)

    #general printing of EPOCHS, LOSS, and Learning Rate
    if epoch % 2 == 0:
        current_lr = optimizer.param_groups[0]['lr']
        print(f"Epoch {epoch} | Loss: {train_loss:.4f} | LR: {current_lr:.6f}")

In [ ]:
from sklearn.metrics import roc_auc_score
import numpy as np

def evaluate():

    # Turn on evaluation protocol
    model.eval()
    y_true = []
    y_pred = []


    #no point in keeping track of grads cause it uses memory
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index, batch.batch)
            #however models output negastive to positive values

            # Apply Sigmoid to turn Logits into Probabilities (0 to 1)
            pred = torch.sigmoid(out)

            y_true.append(batch.y.cpu().numpy())
            y_pred.append(pred.cpu().numpy())

    # Concatenate all batches

    # Combining them vertically

    # After concatenation you get a single 2d array not a list
    y_true = np.concatenate(y_true, axis=0)
    y_pred = np.concatenate(y_pred, axis=0)

    # Calculate ROC-AUC for each of the 12 tasks
    roc_scores = []
    for i in range(12):
        # Only evaluate on valid labels (ignore -1)
        valid_mask = y_true[:, i] != -1
        if valid_mask.sum() > 0: #checking to see if there is at least one valid data point

            score = roc_auc_score(y_true[valid_mask, i], y_pred[valid_mask, i])# calculate score
            roc_scores.append(score) #add to list

    return np.mean(roc_scores)

# Test it now
final_score = evaluate()
print(f"🎯 Final Test ROC-AUC: {final_score:.4f}")

In [ ]:
def predict_toxicity(smiles_string):
    # Switch model to "Eval" mode (turns off Dropout)
    model.eval()

    # Convert string to Graph
    # We pass label=None because we don't know the answer yet
    data = smiles_to_pyg(smiles_string, label=None)

    if data is None:
        return "❌ Error: Could not parse molecule."

    # Move data to the same device as the model (GPU or CPU)
    data = data.to(device)

    # Create a "Batch" of size 1
    # The model expects a batch, even if it's just one item
    batch_idx = torch.zeros(data.num_nodes, dtype=torch.long).to(device)

    # Run the Model
    with torch.no_grad():
        logits = model(data.x, data.edge_index, batch_idx)
        probs = torch.sigmoid(logits) # Convert logits to 0-1 probabilities

    return probs.cpu().numpy().flatten()

In [ ]:
# Figuring out what the 12 task names are
task_names = [c for c in df.columns if c not in ['smiles', 'mol_id']]
print(task_names)

In [ ]:
# Create a way to access smiles based on simply using pubchem's API

def get_smiles_from_name(compound_name):
  try:
    compounds = pcp.get_compounds(compound_name, 'name')

    if compounds:
      smiles = compounds[0].canonical_smiles
      return smiles
    else:
      print(f"Compound '{compound_name}' could not be found")
      return None

  except Exception as e:
        print(f"Error connecting to PubChem: {e}")
        return None


def tester(compound_name):
  smiles_str = get_smiles_from_name(compound_name)

  if smiles_str:
    probs = predict_toxicity(smiles_str)

    print(f"\nTesting 🧪 Toxicity Report for {compound_name}...")

    print("-" * 40)
    for i, task in enumerate(task_names):
        risk = probs[i]
        # Highlight high risks in RED
        flag = "🔴 HIGH RISK" if risk > 0.5 else "🟢 Safe"
        print(f"{task:<15} | {risk:.4f} | {flag}") # Need to adjust this so it's readable
  else:
    print("Error with testing toxicity of compound")

tester("tylenol")

In [ ]:
#implementing GNN explainer portion
from torch_geometric.explain import Explainer, GNNExplainer



In [ ]:
import pubchempy as pcp
import matplotlib.pyplot as plt
import networkx as nx
from torch_geometric.utils import to_networkx

def visualize_custom(data, explanation, title):
    # Extract edge importance
    edge_mask = explanation.edge_mask.cpu().detach().numpy()

    # Create NetworkX graph
    G = to_networkx(data, to_undirected=True)

    plt.figure(figsize=(10, 8))
    pos = nx.spring_layout(G, seed=42) # Consistent layout

    # Draw non-important edges (faint)
    nx.draw_networkx_edges(G, pos, alpha=0.2, width=1)

    # Draw nodes
    nx.draw_networkx_nodes(G, pos, node_size=300, node_color='#add8e6')

    # Draw IMPORTANT edges (The "Toxic" Pharmacophore)
    # Threshold: We only show edges that contributed > 60% to the decision
    threshold = 0.6
    high_importance_edges = []

    # Map edge_mask back to NetworkX edges
    # (PyG edge_index and NetworkX edges might differ in order, so we iterate carefully)
    for i, (u, v) in enumerate(data.edge_index.t().cpu().numpy()):
        if edge_mask[i] > threshold:
            # Add edge to list (NetworkX handles undirected duplicates visually)
            high_importance_edges.append((u, v))

    if high_importance_edges:
        nx.draw_networkx_edges(G, pos, edgelist=high_importance_edges,
                               width=4, edge_color='red')

    plt.title(f"Why is {title} Toxic?\n(Red Bonds = High Attention)")
    plt.axis('off')
    plt.show()

def explain_new_molecule(compound_name):
    print(f"Fetching {compound_name} from PubChem...")

    # Get SMILES
    try:
        compounds = pcp.get_compounds(compound_name, 'name')
        if not compounds:
            print("Compound not found.")
            return
        # Use connectivity_smiles to avoid the DeprecationWarning
        smiles = compounds[0].canonical_smiles
    except Exception as e:
        print(f"Connection Error: {e}")
        return

    # Graph Conversion
    data = smiles_to_pyg(smiles)
    if data is None:
        print("Could not featurize molecule.")
        return
    data = data.to(device)

    # Predict
    model.eval()
    batch_idx = torch.zeros(data.num_nodes, dtype=torch.long).to(device)

    logits = model(data.x, data.edge_index, batch_idx)
    probs = torch.sigmoid(logits).detach().cpu().numpy().flatten()

    target_task_idx = probs.argmax()
    highest_risk = probs[target_task_idx]

    print(f"Analysis: {compound_name}")
    print(f"Top Risk: {task_names[target_task_idx]} ({highest_risk:.4f})")

    if highest_risk < 0.5:
        print("✅ Model thinks this is SAFE. Explainer might be noisy.")
    else:
        print("⚠️ Model thinks this is TOXIC. Generating explanation...")

    # 4. Run Explainer
    explanation = explainer(
        x=data.x,
        edge_index=data.edge_index,
        batch=batch_idx,
        index=0,  # <--- fixed: Always 0 because we have 1 graph
        target=torch.tensor([target_task_idx]).to(device) # Focus on the specific toxic task
    )

    # 5. Visualize
    visualize_custom(data, explanation, compound_name)

In [ ]:
# Try a known toxic compound
explain_new_molecule("Chlorpyrifos")